In [1]:
import pybaseball
import pandas as pd


In [2]:
full_data = pd.read_csv('statcast_2015-2024.csv')

C:\Users\declan\AppData\Local\Temp\ipykernel_27992\4270108768.py:1: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  full_data = pd.read_csv('statcast_2015-2024.csv')


In [3]:
full_data['hit_location'].value_counts()

hit_location
2.0    399904
8.0    202907
9.0    182175
7.0    179627
6.0    151593
4.0    136818
5.0    125016
3.0     84207
1.0     48360
Name: count, dtype: int64

In [3]:
#batter and pitcher columns are id's, lets add 2 columns to include their names
#the player_name column already includes pitcher name, so we can just remove that one

batter_names_df = pybaseball.playerid_reverse_lookup(full_data['batter'].to_list(), key_type='mlbam')
pitcher_names_df = pybaseball.playerid_reverse_lookup(full_data['pitcher'].to_list(), key_type='mlbam')

# Combine first and last names
batter_names_df['full_name'] = batter_names_df['name_first'] + ' ' + batter_names_df['name_last']
pitcher_names_df['full_name'] = pitcher_names_df['name_first'] + ' ' + pitcher_names_df['name_last']

# Create dictionaries to map player IDs to full names
batter_names_dict = batter_names_df.set_index('key_mlbam')['full_name'].to_dict()
pitcher_names_dict = pitcher_names_df.set_index('key_mlbam')['full_name'].to_dict()

# Map the full names to the full_data DataFrame
full_data['batter_name'] = full_data['batter'].map(batter_names_dict)
full_data['pitcher_name'] = full_data['pitcher'].map(pitcher_names_dict)


if 'player_name' in full_data.columns:
    full_data.drop(columns=['player_name'], inplace=True)

In [4]:
#drop depricated cols

deprecated_columns = ['spin_rate_deprecated', 'break_angle_deprecated', 'break_length_deprecated', 'tfs_deprecated', 'tfs_zulu_deprecated', 'spin_dir', 'umpire']
full_data.drop(columns=deprecated_columns, inplace=True)

In [5]:
#drop dups
full_data.drop_duplicates(inplace=True)

In [13]:
#I'll handle missing values in the data processor since the way this works is subject to change

full_data.to_csv('statcast_2015-2024.csv', index=False)

In [14]:
full_data['events'].value_counts()

events
field_out                       686680
strikeout                       384265
single                          247525
walk                            138476
double                           76992
home_run                         54126
force_out                        34671
grounded_into_double_play        32995
hit_by_pitch                     18287
field_error                      12154
sac_fly                          11124
triple                            7295
sac_bunt                          6875
double_play                       3907
fielders_choice                   3514
fielders_choice_out               2832
caught_stealing_2b                1928
intent_walk                       1923
strikeout_double_play             1254
catcher_interf                     447
other_out                          319
sac_fly_double_play                165
caught_stealing_3b                 117
pickoff_1b                         105
caught_stealing_home                84
wild_pitch        